# Pool-Detection

The aim of this project is to provide a POC (Proof Of Concept) to prove the feasibility of **detecting pools from satellite images**.

This task will be divided into two steps:

- Create an image classification model (Convnet) to distinguish between images with pools and images without *no_pools*.
- Then use this model to scan satellite images and find the position of potential pools on the image.

***

## Available Data

```tree
└───data
    ├───train
    │   ├───no_pools
    │   └───pools
    ├───validation
    │   ├───no_pools
    │   └───pools
    └───zones
```

To create the image classifier, we dispose of around 1500 images of each class (pools, no_pools). This data are divided into **train** and **validation** sets. The **train** and **validation** set contain respectively around 1400 and 100 images of each class. Each image has a dimension of 50 x 50.

Sample images:

*pools* : ![alt text](./data/train/pools/img0.jpg)

*no_pools* : ![alt text](./data/train/no_pools/img0.jpg)

As for the satellite images on which the detection of *pools* will be performed, we have 30 of them in the **data/zones/** directory.


Sample satellite image *(zone18.jpg)*:

![alt text](data/zones/zone18.jpg)*zone18.jpg*

- Size of the training data :
    - *pools* : 1398 (50x50) images
    - *no_pools*: 1325 (50x50) images
- Size of the validation data:
    - *pools*: 179 (50x50) images
    - *no_pools*: 176 (50x50) images

***

## Installing the required python packages

```console
pip install -r requirements.txt
```

***

## Baseline Model

The baseline model is a simple 3-layered Convnet. This model is a simple implementation of a *Convolutional Neural Network* and will be used as reference (in terms of performance) to the rest of the tested models.

![alt text](README/PoolNetBaseline_3.png)*PoolNetBaseline Architecture*

## Detection Mechanisms

For the detection part, we divide the image into a grid of 50x50 patches which give us 512 candidate positions to test per satellite image (see image below). The constraint that was imposed for this project is to process one image in less then 10 seconds, but with our baseline model (which is a very simple convnet) we are able to provide predictions for 30 satellite images in around 35 seconds on a CPU (with only two cores).

![alt text](README/decomp.png)*Satellite image decomposition*

Another post-processing has been implemented to provide better predictions. Since we do not have any priors about the position of pools within the satellite image, we test for adjacency on the boxes that we've detected (horizontally or vertically) within the grid. This case is encountered when the pool image is divided between two patches, so we merge the boxes and recompute the probability for that specific patch (Exp: see image below.).

![alt text](README/merging_adj.png)*Merging adjacent bounding boxes for better location prediction*
***

## Results

### Quality of the classifier

![alt text](README/acc_loss_history_3.png)*Train/Validation Loss/Accuracy*

### Quality of the detection

Sample Detection image *(zone18.jpg)*:

![alt text](README/pooldetection_th%3D0.5_zone18.jpg)*Detection on a satellite image with a threshold of 0.5 (the default output of the **detect** class)*

![alt text](README/pooldetection_th%3D0.75_zone18.jpg)*Detection on a satellite image with a threshold of 0.75 on the probability of each patch*

Along with the image we provide a dictionary that contains all information relative to the position and probabilities of each bounding boxes (In this context we only keep the patches with a probability > 0.5, as the purpose of this project is to prove the feasibility of such detection.).

Data relative to the detection image above *(zone18.jpg)*:

**"The reason we have only 6 bounding boxes in the detected image above is that we have a applied a filter that will only display patches with probability >= 0.75. This is done only for purpose of testing and visualization, by default the **detect** class will display all potential pool patches. (ie: probability >= 0.5)"**

```json
"./data/zones/zone18.jpg": {
        "pos": [
            [250,100],
            [1400,200],
            [1100,250],
            [900,500],
            [850,550],
            [1450,650],
            [975,50]
        ],
        "probas": [
            0.9920390248298645,
            0.7856220006942749,
            0.5162228345870972,
            0.9985809922218323,
            0.878549337387085,
            0.9921371936798096,
            0.9999746084213257
        ],
        "nbPools": 7
}
```

Each tuple in the *pos* list represents the (x,y) coordinates of the top left corner of the bounding box that represents a potential pool(to get the center of the bounding box, we just add 25 to x and y as we use a 50x50 sliding window for the detection.). For better detection, we try to improve the predicted position of the bounding boxes by merging two adjacent boxes as sometimes a *pool* could be in two sliding window. This approach helps to make better detections and get closer to the real number of pools in that snapshot.

### Heatmaps

The use of the heatmap is primarily to understand the behaviour of the implemented model and helps make the detection of potential targets easier. We use the probabilities we predicted in the previous task then scale the probability matrix to the size of the input image so we can overlay the resulting heatmap (generated from the probability matrix) onto the image. We use a ```fading_agent``` which is a matrix that creates the fading effect on the heatmap, this is used to indicate the center of the detection as the potential position of the pool.

![alt text](predictions/images/heatmaps/heatmap_zone18.jpg)*Example of a heatmap overlayed on satellite image*

***

## Going Further

This work is considered a Proof of Concept to prove the feasibility of the task and is intended to be a baseline work that can be later improved to create a better detection model.

In this section, I will propose three other approaches that require more data and more preparation:

1. Mine for more data from satellite image APIs or online maps to create more robust data and create an extensive database that includes the position of pools on each image. Then use this data to train an Object Detection model. (exp: ```Fast R-CNN```)

2. Use existing state of the art model for image classification like: ```Resnet```. But from the tests that I conducted during the preparation of this project, these model perform poorly (compared to simple ```Convnets```) on **low resolution** images. For this purpose, the use of a ```SRGAN (Super Resolution GAN)``` to upscale the training images to a size where we can use the aforementioned models without the degradation of performance can be a useful step better results. (A. Upscale the train images B. Train the model (transfer learning) on these images C. Scale the satellite images or use images with higher resolution D. Detect! = *same workflow*)

3. There are plenty of research that have been done on low resolution image recognition that can be implemented for this case as we are dealing with very low resolution images (50 x 50). Since these models are optimized to handle this type of input, we can *expect* better results compared to simple ```Convnet```.

### Credits
- [[1] Keras SR-GAN: Enhancing low resolution images by applying deep networks with adversarial networks (Generative Adversarial Networks) 
to produce high resolutions images.](https://github.com/deepak112/Keras-SRGAN)
- [Blog Keras for the necessary documentation on Keras](https://blog.keras.io/)

# Requirements
```console
keras==2.2.4
tensorflow==1.15.0
numpy==1.16.5
matplotlib==3.1.1
seaborn==0.9.0
pydot==1.4.1
graphiz==2.38
```
**graphiz** and **pydot** are only necessary to the `plot_model` method.

## Creating a baseline Convnet for Pool Detection

In [ ]:
# %load PoolNet.py
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Input, AveragePooling2D
from keras.layers import Activation, Dropout, Dense, Flatten
from keras import applications
from keras.utils import plot_model
from keras import backend as K

class PoolNetBaseline:
    # this is a baseline, a simple convnet model used as
    # a proof of concept
    @staticmethod
    def build(width:int, height:int, channels:int)->Sequential:
        if K.image_data_format() == 'channels_first':
            inputShape = (channels, width, height)
        else:
            inputShape = (width, height, channels)
        
        model = Sequential()
        model.add(Conv2D(32, (3,3), input_shape=inputShape))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (5,5))) # extra
        model.add(Activation('relu')) # extra
        model.add(MaxPooling2D(padding='same'))

        model.add(Conv2D(32, (3,3)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (5,5))) # extra
        model.add(Activation('relu')) # extra
        model.add(MaxPooling2D(padding='same'))

        model.add(Conv2D(32, (3,3)))
        model.add(Activation('relu'))
        model.add(Conv2D(32, (5,5))) # extra
        model.add(Activation('relu')) # extra
        model.add(MaxPooling2D(padding='same'))

        # at this stage the model outputs a 3D feature map of the image

        model.add(Flatten()) # converts the 3D feature map into 1D feature vector
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.1)) # added to limit overfitting
        model.add(Dense(1))
        model.add(Activation('sigmoid')) # return a probability for the True class
        # in quality of classification
        # (Pool), forces the output to be in [0,1]
        # print(model.summary())
        # now return the neural network
        return model

# Leveraging pre-trained model from keras.applications
# Performs poorly on low res images
class PoolNetResnet:
    @staticmethod
    def build(width:int, height:int, channels:int)->Sequential:
        if K.image_data_format() == 'channels_first':
            inputShape = (channels, width, height)
        else:
            inputShape = (width, height, channels)
        inp = Input(shape=inputShape)
        model = applications.vgg16.VGG16(include_top=False, pooling='avg', input_tensor=inp)
        x = model.layers[-1].output

        x = Dense(64)(x)
        x = Activation('relu')(x)
        x = Dropout(0.5)(x) # added to limit overfitting
        x = Dense(1)(x)
        x = Activation('sigmoid')(x)

        model = Model(inputs=inp, outputs=x)

        for i in range(len(model.layers)-2):
            model.layers[i].trainable = False
        # print(model.summary())
        return model

## Defining the data generator and the transformation to be done on input images for a more robust model

In [ ]:
# %load Train.py
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from matplotlib import pyplot as plt
from PoolNet import PoolNetBaseline
import warnings
import os

warnings.filterwarnings("ignore")

class TrainModel:
    def __init__(self, model:Sequential, saveFile:str='best_weights_baseline.h5', *args):
        self.CHECKPOINTPATH = './predictions/weights/'+saveFile
        self.VALDIR = './data/validation'
        self.TRAINDIR = './data/train'
        self.IMGWIDTH, self.IMGHEIGHT = 50, 50
        self.CHANNELS = 3
        self.class1 = '/pools'
        self.class2 = '/no_pools'

        self.graphPath = './predictions/history/acc_loss_history.png'
        self.batchSize = 16 # default values will be changed later
        self.nbEpochs = 50 # default values will be changed later
        self.nbValidation = len([1 for f in os.listdir(self.VALDIR+self.class1) 
            if os.path.isfile(os.path.join(self.VALDIR+self.class1, f))]) + \
            len([1 for f in os.listdir(self.VALDIR+self.class2) 
                if os.path.isfile(os.path.join(self.VALDIR+self.class2, f))]) 
        self.nbTrain = len([1 for f in os.listdir(self.TRAINDIR+self.class1) 
                if os.path.isfile(os.path.join(self.TRAINDIR, f))]) + \
            len([1 for f in os.listdir(self.TRAINDIR+self.class2) 
                if os.path.isfile(os.path.join(self.TRAINDIR+self.class2, f))])
        
        self.trainGenerator = None
        self.valGenerator = None
        self.modelHistory = None

        self.model = model
    
    def dataAugmentation(self)->None:
        # Data augmentation: Transformation to perform on input
        # images to make the classifier more robust to noisier
        # and lower quality input images that might be given during
        # the testing phase

        # Normal data augmentation
        trainDataGen = ImageDataGenerator(
            rotation_range=90,  #default value 180
            width_shift_range=0.3,
            height_shift_range=0.3,
            shear_range=0.3,
            rescale=1./255,
            zoom_range=0.3, # default value 0.2
            horizontal_flip=True,
            fill_mode='nearest'
        )
        # Aggressive data augmentation
        # trainDataGen = ImageDataGenerator(
        #     rotation_range=180,
        #     width_shift_range=0.5,
        #     height_shift_range=0.5,
        #     shear_range=0.2,
        #     rescale=1./255,
        #     zoom_range=0.3,
        #     horizontal_flip=True,
        #     fill_mode='nearest'
        # )
        # For the validation process the only transformation to make is
        # the rescaling of the imag to avoid altering the input
        valDataGen = ImageDataGenerator(
            rescale=1./255
        )

        self.trainGenerator = trainDataGen.flow_from_directory(
            self.TRAINDIR,
            target_size=(self.IMGHEIGHT, self.IMGWIDTH),
            batch_size=self.batchSize,
            class_mode='binary'
        )
        self.valGenerator = valDataGen.flow_from_directory(
            self.VALDIR,
            target_size=(self.IMGHEIGHT, self.IMGWIDTH),
            batch_size=self.batchSize,
            class_mode='binary'
        )
        return


    def trainModel(self, useAutoAugmentation:bool=True, trackPerf:bool=True)->None:
        if useAutoAugmentation:
            self.dataAugmentation()
        # Keras Callbacks:
        # EarlyStopping: Used to stop the training when there are no more gains
        #   in terms of loss (decreasing) and accuracy (increasing)
        # ModelCheckpoint: Used to save the weights of the best performing model
        #   on the validation dataset
        callbacks = [
            ModelCheckpoint(self.CHECKPOINTPATH, monitor='val_loss', save_best_only=True),
            EarlyStopping(monitor='val_loss', patience=5)
        ]

        self.model.compile(
            loss='binary_crossentropy',
            optimizer='sgd',
            metrics=['accuracy']
        )
        self.modelHistory = self.model.fit_generator(
            self.trainGenerator,
            steps_per_epoch=self.nbTrain // self.batchSize,
            epochs=self.nbEpochs,
            validation_data=self.valGenerator,
            validation_steps=self.nbValidation // self.batchSize,
            callbacks=callbacks
        )
        if trackPerf:
            self.__trackModel__()

        return


    def __trackModel__(self)->None:
        # Plot the evolution of the loss and accuracy for training and validation
        # through the epochs
        epochs = range(len(self.modelHistory.history['acc']))
        _, ax1 = plt.subplots()
        ax1.plot(epochs, self.modelHistory.history['loss'], label='train_loss', color='blue')
        ax1.plot(epochs, self.modelHistory.history['val_loss'], label='val_loss', color='red')
        ax1.set_xlabel("# Epochs")
        ax1.set_ylabel('Loss')
        ax1.legend()

        ax2 = ax1.twinx()
        ax2.plot(epochs, self.modelHistory.history['acc'], label='train_acc', color='green')
        ax2.plot(epochs, self.modelHistory.history['val_acc'], label='val_acc', color='orange')
        ax2.set_ylabel('Accuracy')
        ax2.legend()

        plt.title('Loss and Accuracy during the Learning Process')
        plt.savefig(self.graphPath)
        return

## The detection phase: applying the model on the satellite images, creating the bounding boxes, fine tuning the predictions and plotting the images with the bounding boxes and corresponding probabilities

In [ ]:
# %load Detect.py
from keras.models import load_model, Sequential
from keras.preprocessing.image import img_to_array, array_to_img, load_img
from matplotlib import pyplot as plt
from matplotlib import patches as patches
import seaborn as sns
import numpy as np
import json
import time
import warnings
import os

sns.set()
sns.set_style("whitegrid", {'axes.grid' : False})
warnings.filterwarnings("ignore")

class Detector:
    def __init__(self, model:Sequential, zonesPath:str):
        self.model = model
        self.zonesPath = zonesPath
        self.zones = [os.path.join(zonesPath, f) for f in os.listdir(zonesPath) 
            if os.path.isfile(os.path.join(zonesPath, f))]
        self.threshold = 0.5
        
        self.results = {}
    
    def removePrefix(self, s:str)->str:
        if s.startswith(self.zonesPath):
            return s[len(self.zonesPath):]
        return s

    def splitImage(self, img:list)->list:
        # Splits an image of size `satHeight x satWidth` into a list of subimages of size = (imgHeight,imgWidth)
        return [img[:, imgWidth*row:imgWidth*(row+1),imgHeight*col:imgHeight*(col+1),:]
            for row in range(satHeight//imgHeight) for col in range(satWidth//imgWidth)]

    def prepImages(self)->np.ndarray:
        # Splits all the images into `imgHeight x imgWidth` subimage of size = (imgHeight,imgWidth)
        # so we get one np.array for all satellite image to test (for easier and faster predictions
        # of the probabilities)
        imgs = []
        for z in self.zones:
            img = (img_to_array(load_img(z)) * (1./255))
            img = img.reshape((1,) + img.shape)
            imgs.extend(self.splitImage(img))
        return np.array(imgs).reshape((len(imgs),imgHeight,imgWidth,channels))

    def predictProba(self)->tuple:
        x = self.prepImages()
        probas = self.model.predict_proba(x, batch_size=batchSize).reshape(len(x))
        probas = probas.reshape((len(self.zones), (satHeight//imgHeight)*(satWidth//imgWidth)))
        return np.where(probas>=self.threshold, probas, 0.), dict(zip(self.zones, probas))

    def getAdjacentBoxes(self, coords:list)->list:
        # naive approach, this can be further optimized for high resolution
        # images with a high number of detected instances 
        adj = []
        for i in range(len(coords)):
            for j in range(i+1,len(coords)):
                q = self.adjacent(coords[i], coords[j])
                if  q:
                    adj.append([i,j,q])
                else:
                    continue
        return adj

    def adjacent(self, box1:tuple, box2:tuple)->bool:
        if box1[1]==box2[1] and (box1[0]+50 == box2[0] or box1[0]-50 == box2[0]) or \
            box1[0]==box2[0] and (box1[1]+50 == box2[1] or box1[1]-50 == box2[1]):
            return True
        return False

    def cleanProba(self, probas:np.ndarray)->dict:
        res = {}
        # add a test to detect two adjacent boxes
        # if y-adjacent center the boxes on the y-axis
        # if x-adjacent center the boxes on the x-axis
        for idx, p in enumerate(probas):
            p = p.reshape(satHeight//imgHeight, satWidth//imgWidth)
            x,y = np.nonzero(p)
            res[self.zones[idx]] = {
                "pos":list(zip(y*50,x*50)),
                "probas":[ float(p[row][col]) for row,col in list(zip(x,y))],
                "adjacentBoxes":self.getAdjacentBoxes(list(zip(y*50,x*50)))
                # check for adjacent boxes for better detection
            }
            # find adjacent boxes
            toRemove = {
                "pos":[],
                "probas":[]
            }
            predAgain = []
            for bb in res[self.zones[idx]]["adjacentBoxes"]:
                # get old values
                b1 = res[self.zones[idx]]["pos"][bb[0]]
                b2 = res[self.zones[idx]]["pos"][bb[1]]
                # stage for for modifications and new probabilities
                toRemove["pos"].extend([bb[0],bb[1]])
                toRemove["probas"].extend([bb[0],bb[1]])
                # newly better placed box
                newBox = ((b1[0]+b2[0])//2, (b1[1]+b2[1])//2)
                predAgain.append(newBox)
            res[self.zones[idx]]["probas"] = [float(res[self.zones[idx]]["probas"][i])
                for i in range(len(res[self.zones[idx]]["probas"])) if i not in set(toRemove["probas"])]
            res[self.zones[idx]]["pos"] = [(int(res[self.zones[idx]]["pos"][i][0]),int(res[self.zones[idx]]["pos"][i][1]))
                for i in range(len(res[self.zones[idx]]["pos"])) if i not in set(toRemove["pos"])]
            # compute the probability for the new box
            # needs to be optimized!!!
            image = (img_to_array(load_img(self.zones[idx]))*1./255)
            image = image.reshape((1,)+image.shape)
            for h,w in predAgain:
                x = image[:,w:w+50,h:h+50,:]
                pr = self.model.predict_proba(x)[0][0]
                if pr >= self.threshold:
                    res[self.zones[idx]]["pos"].append((int(h),int(w)))
                    res[self.zones[idx]]["probas"].append(float(pr))   
            res[self.zones[idx]]["nbPools"] = len(res[self.zones[idx]]["pos"])
            res[self.zones[idx]].pop("adjacentBoxes",None)
        self.results = res
        return res

    def drawBoxes(self)->None:
        for img in self.results:
            _, ax = plt.subplots(figsize=(20,10))
            ax.imshow(load_img(img))
            imgData = self.results[img]
            # draw the bounding boxes and the probability for each
            # classification
            for coord, proba in zip(imgData["pos"], imgData["probas"]):
                color = "cyan"
                if proba >= 0.85: color = "lime"
                if proba <0.75: color="crimson"
                # drawing the bounding boxes
                rect = patches.Rectangle(coord,imgWidth, imgHeight,
                    linewidth=2, edgecolor=color, facecolor=color, alpha=0.4)
                ax.add_patch(rect)
                # drawing the probabilities for each classification
                if coord[1]+60 >= satHeight:
                    xy = (coord[0],coord[1]-10)
                else:
                    xy = (coord[0],coord[1]+60)
                
                ax.annotate(
                    s="prob:{:.3f}".format(proba),
                    xy=xy, color=color,weight="bold", ha="center", va="center")
            plt.xticks([])
            plt.yticks([])
            plt.savefig("./predictions/images/pooldetection_th={}_{}".format(self.threshold, self.removePrefix(img)))
        return

    def drawHeatmap(self, probaMap:list)->None:
        # cmap = sns.cubehelix_palette(light=1, as_cmap=True, reverse=False)
        cmap = sns.color_palette("Paired")
        # not the best implementation for it
        def f(i:int,j:int)->float:
            # fading function
            fadingRate = 1./(20*np.sqrt(2)) # 24*np.sqrt(2) default value
            # fadingRate = 0 # Debugging Value
            return max([1 - fadingRate * (np.sqrt((24-i)**2+(24-j)**2)),0])
        # the fadingAgent is used to create the fading effect on the heatmap
        fadingAgent = np.array([[f(i,j) for i in range(50)] for j in range(50)])
        for img in self.zones:
            # Loading the image on which we will overlay the heatmap
            _, ax = plt.subplots(figsize=(20,10))
            image = load_img(img)
            ax.imshow(image)
            # Potential targets to mark the potential positions of pools
            potTargets = [[],[]] # [xs:list, ys:list]
            tmpMap = probaMap[img].reshape(satHeight//imgHeight, satWidth//imgWidth)
            # scale up the probability matrix
            newProbaMap = np.zeros((800,1600))
            for i in range(satHeight//imgHeight):
                for j in range(satWidth//imgWidth):
                    newProbaMap[i*50:(i+1)*50, j*50:(j+1)*50] = tmpMap[i][j] * fadingAgent
                    if tmpMap[i][j]>0.5:
                        potTargets[0].append(j*50 + 24)
                        potTargets[1].append(i*50 + 24)
            # overlay the heatmap
            heatmap = sns.heatmap(newProbaMap, ax=ax, cmap=cmap, linewidths=0.0)
            heatmap.collections[0].set_alpha(0.35)
            # mark the potential positions for the pools in the area
            plt.scatter(potTargets[0], potTargets[1], marker='x', s=40, c='red')
            # discard the x and y labels
            plt.xticks([])
            plt.yticks([])
            # save the new heatmap
            plt.savefig("./predictions/images/heatmaps/heatmap_{}".format(self.removePrefix(img)), dpi=1000)

        return

In [ ]:
# global vars
satWidth, satHeight = 1600, 800
imgWidth, imgHeight = 50, 50
channels = 3
batchSize = 16

# load the model: PoolNet from the h5 file
path2Weights = "./predictions/weights/best_weights_baseline_3.h5"
trainFile = "Train.py"

# check if the weights exist
# load the model if the file exist otherwise train the model
# using the trainHelper method implemented in `Train.py`
if not os.path.isfile(path2Weights):
    print("---Training The Model---")
    os.system("python {}".format(trainFile))
    print("---Model Trained---")

print("---Loading the Model---")
model = load_model(path2Weights)
print("---Model Loaded---")

# Load the satellite images
zonesPath = "./data/zones/"

detect = Detector(model, zonesPath)

tic = time.clock()
probas, probaMap = detect.predictProba()
results = detect.cleanProba(probas)
elapsedTime = time.clock() - tic

# detect.drawBoxes()
detect.drawHeatmap(probaMap)
print("Elapsed Time [s]: {:.3f}".format(elapsedTime))
# with open("predictions/results.json","w") as f:
#     json.dump(results, f, indent=4)

print("END")